<img src="../../Master/NotebookAddons/HydroSARbanner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="5"> <b>Load HyP3 Data for Full Frame Processing</b><img style="padding: 7px" src="../../Master/NotebookAddons/UAFLogo_A_647.png" width="170" align="right"/></font>

<br>
<font size="4"> <b> Franz J Meyer and Alex Lewandowski; University of Alaska Fairbanks </b> <br>
</font>

<font size="3"> This notebook downloads data from an ASF-HyP3 subscription and prepares it for full frame processing with subsequent HydroSAR algorithms. Data are downloaded, extracted, and organized. DEM tiles are extracted and mosaicked to a consistent large-scale product for use in other HydroSAR workflows.
    
<b><u>Complete Full Frame Flood Mapping Sequence:</u></b>

This notebook is the first step in the <b>Full Frame Flood Mapping Sequence</b>, which consists of running the following notebooks in the order given:
<ol>
    <li><b>Load HyP3 Data:</b> Run this notebook (LoadHyP3Data-FullFrame.ipynb)</li>
    <li><b>Calculate HAND layer:</b> Run <a href="./Big_Hand_notebook.ipynb">Big_Hand_notebook.ipynb</a></li>
    <li><b>Perform Full Frame Surface Water Mapping:</b> Run <a href="./HYDRO30Workflow-v1.ipynb">HYDRO30Workflow-v1.ipynb</a></li>
</ol>
</font></font>

<hr>
<font face="Calibri" size="5" color="darkred"> <b>Important Note about JupyterHub</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.</b> </font>


# Importing Relevant Python Packages

<font face="Calibri">
<font size="3">In this notebook we will use the following scientific libraries:
<ol type="1">
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
</font>
<br>
<font face="Calibri" size="3"><b>Our first step is to import them:</b> </font>

In [ ]:
%%capture
import os
import glob
import json # for loads
import shutil
import re


from osgeo import gdal
import numpy as np

from IPython.display import HTML, display, clear_output, Markdown

try:
    from tqdm.auto import tqdm
except:
    !pip install tqdm
    from tqdm.auto import tqdm
    
import asf_notebook as asfn

# Load Your Own Data Stack Into the Notebook

<font face="Calibri">
<font size="3"> This notebook assumes that you've created your own data stack over your personal area of interest using the <a href="https://www.asf.alaska.edu/" target="_blank">Alaska Satellite Facility's</a> value-added product system <a href="http://hyp3.asf.alaska.edu/" target="_blank">HyP3</a>. HyP3 is an environment that is used by ASF to prototype value added products and provide them to users to collect feedback. 

This lab expects <a href="https://media.asf.alaska.edu/uploads/RTC/rtc_atbd_v1.2_final.pdf" target="_blank">Radiometric Terrain Corrected</a> (RTC) image products as input, so be sure to select an RTC process when creating the subscription for your input data within HyP. Prefer a unique orbit geometry **(choose ascending or descending, not both)** to keep geometric differences between images low. 

We will retrieve HyP3 data via the HyP3 API. As both HyP3 and the Notebook environment sit in the <a href="https://aws.amazon.com/" target="_blank">Amazon Web Services (AWS)</a> cloud, data transfer is quick and cost effective.</font> 
</font>

<hr>
<font face="Calibri" size="3"> To download data from ASF, you need to provide your <a href="https://www.asf.alaska.edu/get-data/get-started/free-earthdata-account/" target="_blank">NASA Earth Data</a> username to the system. Setup an EarthData account if you do not yet have one. <font color='rgba(200,0,0,0.2)'><b>Note that EarthData's End User License Agreement (EULA) applies when accessing the Hyp3 API from this notebook. If you have not acknowleged the EULA in EarthData, you will need to navigate to <a href="https://earthdata.nasa.gov/" target="_blank">EarthData's home page</a> and complete that process.</b></font>
<br><br>
<b>Login to Earthdata:</b> </font> 

In [ ]:
login = asfn.EarthdataLogin()

<hr>
<font face="Calibri" size="3"> Before we download anything, create a working directory for this analysis and change into it. 
<br><br>
<b>Select or create a working directory for the analysis:</b></font>

In [ ]:
# Creating a Working Directory for your Project
while True:
    data_dir = asfn.input_path(f"\nPlease enter the name of a directory in which to store your data for this analysis.")
    if os.path.exists(data_dir):
        contents = glob.glob(f'{data_dir}/*')
        if len(contents) > 0:
            choice = asfn.handle_old_data(data_dir, contents)
            if choice == 1:
                shutil.rmtree(data_dir)
                os.mkdir(data_dir)
                break
            elif choice == 2:
                break
            else:
                clear_output()
                continue
        else:
            break
    else:
        os.mkdir(data_dir)
        break

<font face="Calibri" size="3"><b>Change into the analysis directory and create a folder in which to download your RTC products:</b></font>

In [ ]:
# Switchting to Working Directory and Setting Paths
analysis_directory = f"{os.getcwd()}/{data_dir}"
os.chdir(analysis_directory)
print(f"Current working directory: {os.getcwd()}")
rtc_path = "rtc_products"
asfn.new_directory(rtc_path)
products_path = f"{analysis_directory}/{rtc_path}"

<font face="Calibri" size="3"><b>Enter a Group ID if downloading from a group subscription:</b> </font>

In [ ]:
# Enter Group ID if Available - Leave Blank if no Group ID exists

while True:
    group_id = input("Leave blank or enter a Group ID")
    if group_id == '':
        group_id = None
        break
    try:
        int(group_id)
    except ValueError:
        print("ValueError: Enter an integer.")
        pass
    else:
        clear_output()
        print(f"Group ID: {group_id}")
        break

<font face="Calibri" size="3"><b>List subscriptions and select one:</b>
<br>
Note: This will only show enabled/active HyP3 subscriptions.</font>

In [ ]:
# Retrieve and List All Active Subscriptions in your HyP3 Account
subscriptions = asfn.get_hyp3_subscriptions(login, group_id=group_id)
if len(subscriptions) > 0:
    display(Markdown("<text style='color:darkred;'>Note: After selecting a subscription, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:darkred;'>Otherwise, you will simply rerun this code cell.</text>"))
    print('\nSelect a Subscription:')
    subscription_id = asfn.select_parameter(subscriptions)
subscription_id

<font face="Calibri" size="3"><b>Save the selected subscription ID:</b> </font>

In [ ]:
# Save selected Subscription ID
try:
    subscription_id = subscription_id.value.split(':')[0]
    print(subscription_id)
except AttributeError:
    pass
    print("AttributeError: 'str' object has no attribute 'value'")
    print('subscription_id has likely expired.')
    display(Markdown("<text style='color:red;'>Try re-running the previous code cell.</text>"))

<font face="Calibri" size="3"><b>Select a date range of products to download:</b> </font>

In [ ]:
# Select Desired Date Range for your Analysis
display(Markdown("<text style='color:darkred;'>Note: After selecting a date range, you should select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you may simply rerun this code cell.</text>"))
print('\nSelect a Date Range:')
products_info = asfn.get_subscription_products_info(subscription_id, login, group_id=group_id)
#print(products_info)
dates = asfn.get_products_dates_insar(products_info)

date_picker = asfn.gui_date_picker(dates)
date_picker

<font face="Calibri" size="3"><b>Save the selected date range:</b> </font>

In [ ]:
# Save Date Range Information
date_range = asfn.get_slider_vals(date_picker)
date_range[0] = date_range[0].date()
date_range[1] = date_range[1].date()
print(f"Date Range: {str(date_range[0])} to {str(date_range[1])}")

<font face="Calibri" size="3"><b>Gather the names and ids for all products in the subscription:</b></font>

In [ ]:
# Retrieve Granule IDs for all Products in the Subscription
granule_names = asfn.get_subscription_granule_names_ids(subscription_id, login)

<font face="Calibri" size="3"><b>Gather the available paths, flight directions, and download urls for the subscription, inside the selected date range:</b></font>

In [ ]:
# Retrieve Information on Paths, Orbit Directions and Other necessary Information
display(Markdown("<text style='color:darkred;'><text style='font-size:150%;'>This may take some time for large subscriptions...</text></text>"))
product_info = asfn.get_product_info(granule_names, products_info, date_range)
display(Markdown(f"<text style=color:blue><text style='font-size:175%;'>Done.</text></text>"))
paths = list(set(product_info['paths']))
paths.append('All Paths')

<hr>
<font face="Calibri" size="3"><b>Select a path or paths (use shift or ctrl to select multiple paths):</b></font>

In [ ]:
# Select Path or Paths to Pull from Archive
display(Markdown("<text style='color:darkred;'>Note: After selecting a path, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Path:')
path_choice = asfn.select_mult_parameters(paths)
path_choice

<font face="Calibri" size="3"><b>Save the selected flight path/s:</b></font>

In [ ]:
# Record your Path Choice
fp = path_choice.value
if fp:
    if 'All Paths' in fp:
        fp = None
    if fp:
        print(f"Flight Path: {fp}")
    else:
        print('Flight Path: All Paths')
else:
    print("WARNING: You must select a flight path in the previous cell, then rerun this cell.")

<font face="Calibri" size="3"><b>Select an orbit Direction:</b></font>

In [ ]:
# Pick the Desired Orbit Direction
valid_directions = set()
for i, path in enumerate(product_info['paths']):
    if not fp or path in fp:
        valid_directions.add(product_info['directions'][i])
if len(valid_directions) > 1:
    display(Markdown("<text style='color:red;'>Note: After selecting a flight direction, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
    display(Markdown("<text style='color:red;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Flight Direction:')
direction_choice = asfn.select_parameter(valid_directions, 'Direction:')
direction_choice

<font face="Calibri" size="3"><b>Save the selected orbit direction:</b></font>

In [ ]:
# Record your Orbit Direction Choice
direction = direction_choice.value
print(f"Orbit Direction: {direction}")

<font face="Calibri" size="3"><b>Create a list of download_urls within the date range, filtered by orbit direction and flight path:</b> </font>

In [ ]:
# Before Downloading, List the Number of Selected Images
download_urls = []
for i, orbit_dir in enumerate(product_info['directions']):
    if orbit_dir == direction:
        if fp == None or product_info['paths'][i] in fp:
            download_urls.append(product_info['urls'][i])
download_urls.sort()
print(f"There are {len(download_urls)} products to download.")

<font face="Calibri" size="3"><b>Download the products, unzip them into the rtc_products directory, and delete the zip files:</b> </font>

In [ ]:
# Now Pull Selected Images into your OpenSARLab Instance
if asfn.path_exists(products_path):
    product_count = 1
    print(f"\nSubscription ID: {subscription_id}")
    for url in download_urls:
        print(f"\nProduct Number {product_count} of {len(download_urls)}:")
        product_count += 1
        product = url.split('/')[5]
        filename = f"{products_path}/{product}"
        # if not already present, we need to download and unzip products
        if not os.path.exists(filename.split('.zip')[0]):
            print(
                f"\n{product} is not present.\nDownloading from {url}")
            cmd = asfn.get_wget_cmd(url, login)
            !$cmd
            print(f"\n")
            asfn.asf_unzip(products_path, product)
            print(f"product: {product}")
            try:
                os.remove(product)
            except OSError:
                pass
            print(f"\nDone.")
        else:
            print(f"{filename} already exists.")
display(Markdown(f"<text style=color:blue><text style='font-size:150%;'>ALL PRODUCTS DOWNLOADED</text></text>"))

# Migrate VV & VH images into folder ```FullFrames```

In [ ]:
# Some Uitility Functions needed for Extracting the Relevant Information from the downloaded Zip Files
def get_tiff_paths(regex, polarization, pths):
    tiff_paths = []
    for pth in glob.glob(pths):
        tiff_path = re.search(regex, pth)
        if tiff_path:
            tiff_paths.append(pth)
    return tiff_paths

polarization = 'VH and VV'
regex = "\w[\--~]{{5,300}}(_|-){}(v|V|h|H).(tif|tiff)$".format(polarization[0])
tiff_pth = f"{rtc_path}/*/*{polarization[0]}*.tif*"
tiff_paths = get_tiff_paths(regex, polarization, tiff_pth)

In [ ]:
# Extract Relevant Files from Zips and Organize your Data
ff_path = f"{analysis_directory}/FullFrames/"
if not os.path.exists('./FullFrames'):
    asfn.new_directory("FullFrames")
print('Copying VV and VH bands into folder FullFrames ...')
for pth in tqdm(tiff_paths):
    pol=(pth.split('.')[0]).split("_")[-1]
    imdate=asfn.date_from_product_name(pth).split('T')[0]
    imtime=(pth.split('T')[1]).split("_")[0]
    outname = (f"{imdate}T{imtime}_{pol}.tif")
    command = f"cp {pth} {ff_path}/{outname}"
    !$command
    

# Mosaicking DEMs 

## Collect DEM files and Migrate into Folder ```DEMs```

In [ ]:
# Look for DEM files in your Subscription --> Needed to facilitate HAND Calculation
if not os.path.exists('./DEMs'):
    asfn.new_directory("DEMs")
!cp rtc_products/*/*dem.tif DEMs/

## Performing Mosaicking

<font face="Calibri" size="3"><b>Writing Support Functions:</b> </font>

In [ ]:
# Writing some Uitility Functions
def get_DEM_paths(pths):
    DEM_paths = []
    for pth in glob.glob(pths):
        DEM_paths.append(pth)
    return DEM_paths

def get_DEM_dates(paths):
    dates = []
    for pth in tiff_paths:
        date = pth.split("/")[-1].split("_")[3].split("T")[0]
        dates.append(date)
    return dates

def print_tiff_paths(tiff_paths):
    print("Tiff paths:")
    for p in tiff_paths:
        print(f"{p}\n")

<font face="Calibri" size="3"><b>Identifying Predominant UTM zone:</b> </font>

In [ ]:
# Check UTM Zones and Identify Dominant UTM Zone for your Subscription
DEM_pth = f"DEMs/*.tif*"
DEM_paths = get_DEM_paths(DEM_pth)

utm_zones = []
utm_types = []
print('Checking UTM Zones in the DEM data stack ...\n')
for k in range(0, len(DEM_paths)):
    info = (gdal.Info(DEM_paths[k], options = ['-json']))
    info = json.dumps(info)
    info = (json.loads(info))['coordinateSystem']['wkt']
    zone = info.split('ID')[-1].split(',')[1][0:-2]
    utm_zones.append(zone)
    typ = info.split('ID')[-1].split('"')[1]
    utm_types.append(typ)
#print(f"UTM Zones:\n {utm_zones}\n")
#print(f"UTM Types:\n {utm_types}")

# Identifying Predominant UTM Zone

utm_unique, counts = np.unique(utm_zones, return_counts=True)
a = np.where(counts == np.max(counts))
predominant_utm = utm_unique[a][0]
print(f"Predominant UTM Zone: {predominant_utm}")

<font face="Calibri" size="3"><b>Reproject and Mosaic DEMs:</b> </font>

In [ ]:
# Reproject (if needed) and Mosaic DEM Files in Preparation for Subsequent HAND Calculation
reproject_indicies = [i for i, j in enumerate(utm_zones) if j != predominant_utm] #makes list of indicies in utm_zones that need to be reprojected
print('--------------------------------------------')
print('Reprojecting %4.1f files' %(len(reproject_indicies)))
print('--------------------------------------------')
for k in reproject_indicies:
    temppath = DEM_paths[k].strip()
    product_name, DEM_name = temppath.split('/')
    cmd = f"gdalwarp -overwrite {product_name}/{DEM_name} {product_name}/r{DEM_name} -s_srs {utm_types[k]}:{utm_zones[k]} -t_srs EPSG:{predominant_utm}"
    !{cmd}
    rm_command = f"rm {DEM_paths[k].strip()}"
    #print(f"Calling the command: {rm_command}")
    !{rm_command}

print(' ')
print('--------------------------------------------')
print('Updating DEM Paths')
print('--------------------------------------------')
DEM_paths = get_DEM_paths(DEM_pth)
print_tiff_paths(DEM_paths)

print(' ')
print('--------------------------------------------')
print('Update Key Values')
print('--------------------------------------------')    
DEMin = [{}]
d_paths = get_DEM_paths(DEM_pth)
test = 0
for pth in d_paths:
    if test == 0:
        DEMin[0] = f"{pth}"
    else:
        DEMin[0] = f"{DEMin[0]} {pth}"
    test = test + 1

for d in DEMin:
    print(d)
    print("\n")
    
print(' ')
print('--------------------------------------------')
print('Update Key Values')
print('--------------------------------------------')    
output = f"{DEMin[0].split('/')[0]}/DEM-Mosaic.tif"
gdal_command = f"gdal_merge.py -o {output} {DEMin[0]}"
print(f"\n\nCalling the command: {gdal_command}\n")
!{gdal_command}
for pth in DEMin[0].split(' '):
    if pth and asfn.path_exists(pth):
        os.remove(pth)
        print(f"Deleting: {pth}")

# Cleanup Workspace

In [ ]:
# Cleaning up Temporary Files
demcopy = f"cp DEMs/DEM-Mosaic.tif {ff_path}/"
!{demcopy}
!rm -r DEMs/
!rm -r rtc_products/

# Version History

<font face="Calibri" size="2"> <i>LoadHyP3Data-FullFrame.ipynb - Version 1.1.0 - April 2021
    <br>
        <b>Version Changes:</b>
    <ul>
        <li>First Version of this notebook</li> 
    </ul>
    </i>
</font>